In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [2]:
import pandas as pd
from tqdm import tqdm
from qdrant_client import QdrantClient, models
from FlagEmbedding import BGEM3FlagModel, FlagLLMReranker

from src.conf import url_qdrant


tqdm.pandas()

https://github.com/qdrant/workshop-ultimate-hybrid-search/blob/main/notebooks/02-hybrid-search.ipynb

### Анализ работы разных поисков и реранков

In [ ]:
qdrant_client = QdrantClient(url_qdrant)
collection_name = "911_hybrid_rating_points"

In [16]:
def convert_texts(text: list):
    st = ""
    for index, text in enumerate(text):
        st += f"Документ {index+1}"
        st += "\n\n\n"
        st += text
        st += "\n\n\n"
    return st


def search_data(collection_name, point, reranker_bge=None, reranker_gemma=None, n=10):
    vector = point.vector
    question = point.payload["question"]

    # запрос для разряженного вектора
    sparse = models.Prefetch(
        query=models.SparseVector(
            indices=vector["text-sparse"].indices,
            values=vector["text-sparse"].values,
        ),
        using="text-sparse",
        limit=100,
    )

    # запрос для разряженного вектора
    sparse_1000 = models.Prefetch(
        query=models.SparseVector(
            indices=vector["text-sparse"].indices,
            values=vector["text-sparse"].values,
        ),
        using="text-sparse",
        limit=1000,
    )

    # Запрос для плотного вектора
    dense = models.Prefetch(query=vector["dense"], using="dense", limit=100)

    # Запрос для плотного вектора 1000
    dense_1000 = models.Prefetch(query=vector["dense"], using="dense", limit=1000)

    # Запрашиваем 1000 по плотным векторам из них 100 по разряженным
    dence_sparse = models.Prefetch(
        prefetch=[dense_1000],
        query=models.SparseVector(
            indices=vector["text-sparse"].indices,
            values=vector["text-sparse"].values,
        ),
        using="text-sparse",
        limit=100,
    )

    # Запрашиваем 1000 по разряженным векторам из них 100 по плотным
    sparce_dense = models.Prefetch(
        prefetch=[sparse_1000], query=vector["dense"], using="dense", limit=100
    )

    ## Запрашиваем данные

    record = {}
    record["question"] = question

    if reranker_bge or reranker_gemma:
        record["model_rerank_type"] = [reranker_bge.model_name_or_path, reranker_gemma.model_name_or_path]

    for name_search_type, search_type in [
        ("dense", [dense]),
        ("sparse", [sparse]),
        ("sparse+dense", [sparse, dense]),
        ("sparce_dense", [sparce_dense]),
        ("dence_sparse", [dence_sparse]),
    ]:
        point = qdrant_client.query_points(
            collection_name=collection_name,
            prefetch=search_type,
            limit=100,
            query=models.FusionQuery(
                fusion=models.Fusion.RRF,
            ),
            timeout=1000,
        ).points

        texts = [i.payload["question"] for i in point[1:]]
        record[name_search_type] = convert_texts(texts[:n])
        record[f"{name_search_type}_len"] = sum([len(i) for i in texts[:n]])

        if reranker_bge:
            score = reranker_bge.compute_score([[question, i] for i in texts])
            texts_score = sorted(
                [(text, score) for text, score in zip(texts, score)],
                key=lambda x: x[1],
                reverse=True,
            )[:n]
            texts = [i[0] for i in texts_score]
            record[f"{name_search_type}_reranker_bge"] = convert_texts(texts)
            record[f"{name_search_type}_reranker_bge_len"] = sum([len(i) for i in texts])
        
        if reranker_gemma:
            score = reranker_gemma.compute_score([[question, i] for i in texts])
            texts_score = sorted(
                [(text, score) for text, score in zip(texts, score)],
                key=lambda x: x[1],
                reverse=True,
            )[:n]
            texts = [i[0] for i in texts_score]
            record[f"{name_search_type}_reranker_gemma"] = convert_texts(texts)
            record[f"{name_search_type}_reranker_gemma_len"] = sum([len(i) for i in texts])
            
    return record

In [17]:
def get_point(collection_name, n):
    return qdrant_client.query_points(
        collection_name=collection_name,
        query=models.SampleQuery(sample=models.Sample.RANDOM),
        limit=n,
        with_vectors=True,
    ).points

In [18]:
collection_name = "911_hybrid"

points = get_point(collection_name, n=20)
reranker_gemma = FlagLLMReranker("BAAI/bge-reranker-v2-gemma", use_fp16=True)
reranker_bge = FlagLLMReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)

records = []
for point in tqdm(points):
    records.append(search_data(collection_name="911_hybrid", point=point, n=5, reranker_gemma=reranker_gemma, reranker_bge=reranker_bge))

pd.DataFrame(records).to_csv("./data/interim/rag_results/rag_results.csv")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

If you want to use `XLMRobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of XLMRobertaForCausalLM were not initialized from the model checkpoint at BAAI/bge-reranker-v2-m3 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 7/7 [00:00<00:00, 11.12it/s]
You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 20/20 [01:03<00:00,  3.18s/it]


In [20]:
records[0]

{'question': 'Я живу в квартире, принадлежащей мне на праве собственности. Раньше со мной проживала бабушка, которая в апреле 2009 года умерла. В декабре 2009 я узнала, что водоснабжающая организация МУП "Водник" обратилось в службу судебных приставов для взыскании с меня задолженности по исполниельному листу. Для уточнения долга я обратилась в МУП "Водник", выяснилось, что по настоящий момент мне начисляют оплату за водоснабжение за 2 человек, меня и бабушку. Я предоставила свидетельство о смерти бабушки, на которую производилось начисление, но мне отказали в перерасчете, сославшись на то, что я должна была предоставить эти сведения сразу. Помогите разобраться в этой ситуации. Заранее благодарна.',
 'model_rerank_type': ['BAAI/bge-reranker-v2-m3',
  'BAAI/bge-reranker-v2-gemma'],
 'dense': 'Документ 1\n\n\nДосталась квартира в наследство по завещанию от прабабушки. Я прописана и проживаю по другому адресу. Оформление наследства очень затянулось, образовался долг. На данный момент я за

In [21]:
pd.DataFrame(records).to_excel("./data/interim/rag_results/rag_results.xlsx")

In [ ]:
df_records = pd.DataFrame(records)

df_records.describe()

,dense_len,sparse_len,sparse+dense_len,sparce_dense_len,dence_sparse_len
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,1584.780000,44105.685000,28930.470000,1799.600000,3884.205000
std,1336.216425,55871.782774,38865.645246,1445.435397,5791.876304
min,189.000000,229.000000,207.000000,191.000000,229.000000
25%,586.500000,3434.500000,1779.750000,643.500000,1556.000000
50%,1277.000000,16603.000000,5443.500000,1444.000000,2866.000000
75%,2144.000000,68814.500000,43500.500000,2452.000000,4322.750000
max,9111.000000,234500.000000,177439.000000,8457.000000,73822.000000
